In [5]:
import os
import pandas as pd

# Função para juntar todos os arquivos CSV de uma pasta em um único DataFrame
def juntar_csvs(pasta, arquivo_saida):
    arquivos = [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith('.csv')]
    dfs = [pd.read_csv(f) for f in arquivos]
    df_final = pd.concat(dfs, ignore_index=True)
    df_final.to_csv(arquivo_saida, index=False)
    print(f"Arquivo salvo: {arquivo_saida}")

# Juntar arquivos de comments
juntar_csvs('dataset/comments', 'dataset/comments.csv')

# Juntar arquivos de subreddits
juntar_csvs('dataset/subreddits', 'dataset/subreddits.csv')

Arquivo salvo: dataset/comments.csv
Arquivo salvo: dataset/subreddits.csv


In [7]:
import pandas as pd
import re
import os

print("Carregando os datasets...")

# Defina os nomes dos seus arquivos aqui
posts_file = 'dataset/subreddits.csv'
comments_file = 'dataset/comments.csv'

# Carrega os arquivos CSV para DataFrames do pandas
try:
    df_posts = pd.read_csv(posts_file)
    df_comments = pd.read_csv(comments_file)
    print(f"Carregados com sucesso!")
    print(f"Posts: {len(df_posts)} linhas")
    print(f"Comentários: {len(df_comments)} linhas")
except FileNotFoundError:
    print(f"Erro: Arquivos não encontrados. Verifique se os nomes '{posts_file}' e '{comments_file}' estão corretos e na pasta 'data'.")

Carregando os datasets...
Carregados com sucesso!
Posts: 6687 linhas
Comentários: 61472 linhas


In [8]:
# 1. Removendo posts duplicados
posts_originais = len(df_posts)
df_posts.drop_duplicates(subset=['id'], inplace=True)
print(f"Removidos {posts_originais - len(df_posts)} posts duplicados.")

# 2. Função para remover emojis
def remove_emojis(text):
    if not isinstance(text, str):
        return text
    # Regex que encontra a maioria dos emojis e outros símbolos
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE,
    )
    return emoji_pattern.sub(r'', text)

print("\nLimpando emojis dos títulos e corpos dos posts e comentários...")
df_posts['title'] = df_posts['title'].apply(remove_emojis)
df_posts['body'] = df_posts['body'].apply(remove_emojis)
df_comments['body'] = df_comments['body'].apply(remove_emojis)

print("Limpeza de emojis concluída.")
df_posts.head()

Removidos 0 posts duplicados.

Limpando emojis dos títulos e corpos dos posts e comentários...
Limpeza de emojis concluída.


,id,title,body,author,score,upvote_ratio,num_comments,created_utc,subreddit,keyword,method,time_filter,year,month
0,11a27vu,eu_nvr,NaN,umapessoaonline,7948,1.00,79,1.677171e+09,eu_nvr,GPT,praw_search_all,all,2023,2
1,14uyust,Eu_nvr,NaN,Priffos,2693,0.99,98,1.688910e+09,eu_nvr,Algoritmos de IA,praw_search_all,all,2023,7
2,1hmq4rw,Eu🤖nvr,NaN,Murilo_Br23,2278,0.99,47,1.735226e+09,eu_nvr,GPT,praw_search_all,all,2024,12
3,1glbrnm,eu_nvr,NaN,RevolutionaryWhale,2216,1.00,15,1.730934e+09,eu_nvr,Transformers,praw_search_all,all,2024,11
4,1087c0v,Eu_nvr,NaN,LordTaw,2065,0.98,48,1.673349e+09,eu_nvr,Algoritmos de IA,praw_search_all,all,2023,1


In [9]:
print("Iniciando a filtragem de posts com corpo vazio e sem comentários...")

# Primeiro, contamos quantos comentários cada post tem
comment_counts = df_comments['post_id'].value_counts().reset_index()
comment_counts.columns = ['post_id', 'comment_count']

# Agora, juntamos essa contagem de volta ao DataFrame de posts
# Usamos 'left' para manter todos os posts, mesmo os que não têm comentários (eles ficarão com NaN)
df_posts = pd.merge(df_posts, comment_counts, left_on='id', right_on='post_id', how='left')

# Preenchemos os posts sem comentários com 0
df_posts['comment_count'].fillna(0, inplace=True)
df_posts['comment_count'] = df_posts['comment_count'].astype(int)

# Preenchemos corpos vazios (NaN) com uma string vazia para a verificação
df_posts['body'].fillna('', inplace=True)

# Armazenamos o número de posts antes de filtrar
total_antes_filtro = len(df_posts)

# Aplicamos o filtro: selecionamos os posts que NÃO atendem à condição de remoção
# Condição para manter: (corpo NÃO é vazio) OU (contagem de comentários > 0)
filtro_manter = (df_posts['body'].str.strip() != '') | (df_posts['comment_count'] > 0)
df_posts_filtrado = df_posts[filtro_manter].copy()

# Remove as colunas auxiliares que não precisamos mais
df_posts_filtrado.drop(columns=['post_id', 'comment_count'], inplace=True)

print(f"\nFiltro concluído!")
print(f"Número de posts antes da filtragem: {total_antes_filtro}")
print(f"Número de posts após a filtragem: {len(df_posts_filtrado)}")
print(f"Total de {total_antes_filtro - len(df_posts_filtrado)} posts removidos (corpo vazio e sem comentários).")

Iniciando a filtragem de posts com corpo vazio e sem comentários...

Filtro concluído!
Número de posts antes da filtragem: 6687
Número de posts após a filtragem: 6657
Total de 30 posts removidos (corpo vazio e sem comentários).


In [11]:
# Criar o diretório de saída se não existir
output_dir = 'dataset/limpos'
os.makedirs(output_dir, exist_ok=True)

# Nomes dos novos arquivos
posts_limpos_file = os.path.join(output_dir, 'posts_limpos.csv')
comments_limpos_file = os.path.join(output_dir, 'comments_limpos.csv')

print(f"Salvando arquivos limpos em '{output_dir}'...")

# Salva os dataframes
df_posts_filtrado.to_csv(posts_limpos_file, index=False, encoding='utf-8-sig')
df_comments.to_csv(comments_limpos_file, index=False, encoding='utf-8-sig')

print(f"Posts limpos salvos em: '{posts_limpos_file}'")
print(f"Comentários (com emojis removidos) salvos em: '{comments_limpos_file}'")

Salvando arquivos limpos em 'dataset/limpos'...
Posts limpos salvos em: 'dataset/limpos/posts_limpos.csv'
Comentários (com emojis removidos) salvos em: 'dataset/limpos/comments_limpos.csv'


In [13]:
import pandas as pd
import os
from tqdm.auto import tqdm

# Configura o pandas para não truncar a exibição das colunas de texto
pd.set_option('display.max_colwidth', None)
# Registra o tqdm para poder usar o .progress_apply()
tqdm.pandas()

print("Carregando os datasets limpos...")

# Caminhos para os arquivos gerados no passo anterior
posts_file = 'dataset/limpos/posts_limpos.csv'
comments_file = 'dataset/limpos/comments_limpos.csv'

try:
    df_posts = pd.read_csv(posts_file)
    df_comments = pd.read_csv(comments_file)
    print("Arquivos carregados com sucesso!")
    print(f"Posts para análise: {len(df_posts)}")
    print(f"Comentários para análise: {len(df_comments)}")
except FileNotFoundError:
    print(f"Erro: Arquivos não encontrados. Verifique se os arquivos '{posts_file}' e '{comments_file}' existem.")

Carregando os datasets limpos...
Arquivos carregados com sucesso!
Posts para análise: 6657
Comentários para análise: 61472


In [14]:
print("Otimizando a estrutura de comentários para busca rápida...")

# Garante que não há valores nulos no corpo dos comentários
df_comments['body'].fillna('', inplace=True)

# Agrupa os comentários por post_id e junta todos os corpos em uma única string
# Usamos .lower() aqui para já deixar o texto pronto para a busca case-insensitive
comments_grouped = df_comments.groupby('post_id')['body'].apply(lambda x: ' '.join(x).lower())

# Converte a Série agrupada para um dicionário para acesso O(1) (muito rápido)
comments_dict = comments_grouped.to_dict()

print("Estrutura de comentários pronta para verificação.")

Otimizando a estrutura de comentários para busca rápida...
Estrutura de comentários pronta para verificação.


In [16]:
def verificar_relevancia(row):
    """
    Verifica se um post é relevante.
    Retorna True se a keyword exata está no post ou em seus comentários.
    """
    # Pega os dados da linha (do post)
    post_id = row['id']
    keyword = str(row['keyword']).lower() # Keyword do post em minúsculo
    
    # Concatena título e corpo do post, tratando valores nulos e convertendo para minúsculo
    titulo = str(row['title']).lower() if pd.notna(row['title']) else ''
    corpo_post = str(row['body']).lower() if pd.notna(row['body']) else ''
    texto_post = titulo + ' ' + corpo_post
    
    # 1. Verifica se a keyword exata está no próprio post
    if keyword in texto_post:
        return True
    
    # 2. Se não estiver, busca a keyword em todos os comentários associados ao post
    if post_id in comments_dict:
        texto_comentarios = comments_dict[post_id]
        if keyword in texto_comentarios:
            return True
            
    # 3. Se não encontrou em nenhum lugar, o post não é relevante
    return False

In [17]:
print("Aplicando o filtro de relevância em todos os posts. Isso pode levar alguns minutos...")

# Aplica a função para criar uma nova coluna booleana ('relevante')
df_posts['relevante'] = df_posts.progress_apply(verificar_relevancia, axis=1)

# Filtra o DataFrame de posts para manter apenas as linhas marcadas como relevantes
df_posts_final = df_posts[df_posts['relevante'] == True].copy()

# Remove a coluna auxiliar 'relevante'
df_posts_final.drop(columns=['relevante'], inplace=True)

print("\n--- Resultados da Filtragem ---")
total_antes = len(df_posts)
total_depois = len(df_posts_final)
print(f"Número de posts antes da filtragem final: {total_antes}")
print(f"Número de posts após a filtragem final (super filtrado): {total_depois}")
print(f"Total de {total_antes - total_depois} posts removidos por falta de relevância.")

Aplicando o filtro de relevância em todos os posts. Isso pode levar alguns minutos...


100%|██████████| 6657/6657 [00:00<00:00, 20552.50it/s]


--- Resultados da Filtragem ---
Número de posts antes da filtragem final: 6657
Número de posts após a filtragem final (super filtrado): 1202
Total de 5455 posts removidos por falta de relevância.


In [18]:
# Pega a lista de IDs dos posts que foram mantidos
ids_relevantes = df_posts_final['id'].unique()

# Filtra o DataFrame de comentários original para manter apenas os relevantes
df_comments_final = df_comments[df_comments['post_id'].isin(ids_relevantes)].copy()

# --- Salvar os arquivos finais ---
output_dir = 'data/super_filtrado'
os.makedirs(output_dir, exist_ok=True)

posts_final_file = os.path.join(output_dir, 'posts_super_filtrado.csv')
comments_final_file = os.path.join(output_dir, 'comments_super_filtrado.csv')

print(f"\nSalvando datasets super filtrados em '{output_dir}'...")

df_posts_final.to_csv(posts_final_file, index=False, encoding='utf-8-sig')
df_comments_final.to_csv(comments_final_file, index=False, encoding='utf-8-sig')

print(f"Posts finais salvos em: '{posts_final_file}' ({len(df_posts_final)} linhas)")
print(f"Comentários finais salvos em: '{comments_final_file}' ({len(df_comments_final)} linhas)")


Salvando datasets super filtrados em 'data/super_filtrado'...
Posts finais salvos em: 'data/super_filtrado/posts_super_filtrado.csv' (1202 linhas)
Comentários finais salvos em: 'data/super_filtrado/comments_super_filtrado.csv' (8202 linhas)


In [19]:
import pandas as pd
import re
import os
import logging

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminhos para os arquivos gerados na última etapa de filtragem
POSTS_FILE = 'data/super_filtrado/posts_super_filtrado.csv'
COMMENTS_FILE = 'data/super_filtrado/comments_super_filtrado.csv'
OUTPUT_FILE = 'data/dataset_unificado_para_analise.csv'

print("Carregando datasets super filtrados...")
try:
    df_posts = pd.read_csv(POSTS_FILE, dtype={'id': str})
    df_comments = pd.read_csv(COMMENTS_FILE, dtype={'id': str, 'post_id': str})
    print(f"Carregados {len(df_posts)} posts e {len(df_comments)} comentários.")
except FileNotFoundError as e:
    logging.error(f"Erro: Arquivo não encontrado. {e}. Verifique o caminho dos arquivos.")

Carregando datasets super filtrados...
Carregados 1202 posts e 8202 comentários.


In [20]:
logging.info("Iniciando a unificação...")

# --- 1. PREPARAÇÃO DOS POSTS ---
# Cria a coluna de texto principal juntando título e corpo
df_posts['text'] = df_posts['title'].fillna('') + ' ' + df_posts['body'].fillna('')
# Renomeia 'id' para 'doc_id' (ID do Documento) para padronizar
df_posts.rename(columns={'id': 'doc_id'}, inplace=True)
# Cria uma coluna para identificar o tipo de documento
df_posts['doc_type'] = 'post'
# Seleciona as colunas que queremos manter para a análise final
posts_final = df_posts[['doc_id', 'doc_type', 'text', 'created_utc', 'score', 'num_comments', 'subreddit', 'category', 'keyword']]


# --- 2. PREPARAÇÃO DOS COMENTÁRIOS ---
# Renomeia 'id' para 'doc_id' e 'body' para 'text' para compatibilidade
df_comments.rename(columns={'id': 'doc_id', 'body': 'text'}, inplace=True)
# Adiciona a coluna de tipo de documento
df_comments['doc_type'] = 'comment'
# Comentários não têm "num_comments", então preenchemos com 0 para manter a estrutura
df_comments['num_comments'] = 0
# Seleciona as colunas finais, mantendo 'post_id' para rastrear a origem
comments_final = df_comments[['doc_id', 'doc_type', 'text', 'created_utc', 'score', 'num_comments', 'subreddit', 'post_id', 'keyword']]
# Adiciona a coluna 'category' que pode estar faltando nos comentários
# (Se precisar, podemos buscar a categoria do post original)
if 'category' not in comments_final.columns:
    # Mapeia a categoria do post pai para cada comentário
    category_map = df_posts.set_index('doc_id')['category']
    comments_final['category'] = comments_final['post_id'].map(category_map)


# --- 3. CONCATENAÇÃO ---
df_unificado = pd.concat([posts_final, comments_final], ignore_index=True)
logging.info(f"Unificação completa. O dataset final tem {len(df_unificado)} documentos.")

# --- 4. LIMPEZA LEVE FINAL ---
# Aplica a mesma limpeza leve para remover URLs, etc., garantindo consistência
def light_clean(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\[\s*(removido|deletado)\s*\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_unificado['text_cleaned'] = df_unificado['text'].apply(light_clean)
df_unificado = df_unificado[df_unificado['text_cleaned'].str.strip() != ''].copy()

# --- 5. SALVAR ---
os.makedirs('data', exist_ok=True)
df_unificado.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

logging.info(f"Dataset unificado salvo com sucesso em '{OUTPUT_FILE}'. Total de {len(df_unificado)} linhas prontas para a classificação com LLM.")

df_unificado.head()

2025-08-06 18:31:08,460 - INFO - Iniciando a unificação...


KeyError: "['category'] not in index"

In [21]:
import pandas as pd
import re
import os
import logging

# Configuração do logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Caminhos para os arquivos gerados na última etapa de filtragem
POSTS_FILE = 'data/super_filtrado/posts_super_filtrado.csv'
COMMENTS_FILE = 'data/super_filtrado/comments_super_filtrado.csv'
OUTPUT_FILE = 'data/dataset_unificado_para_analise.csv'

print("Carregando datasets super filtrados...")
try:
    df_posts = pd.read_csv(POSTS_FILE, dtype={'id': str})
    df_comments = pd.read_csv(COMMENTS_FILE, dtype={'id': str, 'post_id': str})
    print(f"Carregados {len(df_posts)} posts e {len(df_comments)} comentários.")
except FileNotFoundError as e:
    logging.error(f"Erro: Arquivo não encontrado. {e}. Verifique o caminho dos arquivos.")
# ==============================================================================
#                             INÍCIO DA CORREÇÃO
# ==============================================================================
logging.info("Iniciando a unificação...")

# --- 1.5 RECRIAR A COLUNA 'category' ---
# Definimos aqui o mapeamento original de subreddits para categorias
category_mapping = {
    'geral': ['brasil', 'brasil2', 'conversas', 'PergunteReddit', 'r/BrasilOnReddit'],
    'tecnologia': ['brdev', 'datasciencebr', 'chatgpt_brasil', 'computadores',
                   'WindowsBrasil', 'Aplicativo', 'AssistenciaTecnica',
                   'Programadores_Alados', 'hardwarebrasil', 'Linuxbrasil', 'programacao']
}

# Invertemos o dicionário para mapear cada subreddit à sua categoria
subreddit_to_category = {sub: cat for cat, subs in category_mapping.items() for sub in subs}

# Criamos a coluna 'category' no DataFrame de posts
df_posts['category'] = df_posts['subreddit'].map(subreddit_to_category)
# Preenchemos qualquer valor não mapeado como 'outro' para segurança
df_posts['category'].fillna('outro', inplace=True)

logging.info("Coluna 'category' recriada com sucesso nos posts.")
# ==============================================================================
#                              FIM DA CORREÇÃO
# ==============================================================================

# --- 2. PREPARAÇÃO DOS POSTS ---
df_posts['text'] = df_posts['title'].fillna('') + ' ' + df_posts['body'].fillna('')
df_posts.rename(columns={'id': 'doc_id'}, inplace=True)
df_posts['doc_type'] = 'post'
posts_final = df_posts[['doc_id', 'doc_type', 'text', 'created_utc', 'score', 'num_comments', 'subreddit', 'category', 'keyword']]


# --- 3. PREPARAÇÃO DOS COMENTÁRIOS ---
df_comments.rename(columns={'id': 'doc_id', 'body': 'text'}, inplace=True)
df_comments['doc_type'] = 'comment'
df_comments['num_comments'] = 0

# CORREÇÃO ADICIONAL: Mapeia a categoria do post pai para cada comentário
category_map = df_posts.set_index('doc_id')['category']
df_comments['category'] = df_comments['post_id'].map(category_map)
# Remove comentários cujo post pai não existe mais no df_posts (foram filtrados)
df_comments.dropna(subset=['category'], inplace=True)

comments_final = df_comments[['doc_id', 'doc_type', 'text', 'created_utc', 'score', 'num_comments', 'subreddit', 'post_id', 'keyword', 'category']]


# --- 4. CONCATENAÇÃO ---
df_unificado = pd.concat([posts_final, comments_final], ignore_index=True)
logging.info(f"Unificação completa. O dataset final tem {len(df_unificado)} documentos.")

# --- 5. LIMPEZA LEVE FINAL ---
def light_clean(text):
    if not isinstance(text, str):
        return ""
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\[\s*(removido|deletado)\s*\]', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_unificado['text_cleaned'] = df_unificado['text'].apply(light_clean)
df_unificado = df_unificado[df_unificado['text_cleaned'].str.strip() != ''].copy()

# --- 6. SALVAR ---
os.makedirs('data', exist_ok=True)
df_unificado.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

logging.info(f"Dataset unificado salvo com sucesso em '{OUTPUT_FILE}'. Total de {len(df_unificado)} linhas prontas para a classificação com LLM.")

df_unificado.head()

2025-08-06 18:33:17,801 - INFO - Iniciando a unificação...
2025-08-06 18:33:17,805 - INFO - Coluna 'category' recriada com sucesso nos posts.
2025-08-06 18:33:17,849 - INFO - Unificação completa. O dataset final tem 9404 documentos.


Carregando datasets super filtrados...
Carregados 1202 posts e 8202 comentários.


2025-08-06 18:33:18,337 - INFO - Dataset unificado salvo com sucesso em 'data/dataset_unificado_para_analise.csv'. Total de 9337 linhas prontas para a classificação com LLM.


,doc_id,doc_type,text,created_utc,score,num_comments,subreddit,category,keyword,post_id,text_cleaned
0,11a27vu,post,eu_nvr,1.677171e+09,7948,79,eu_nvr,outro,GPT,NaN,eu_nvr
1,1hmq4rw,post,Eu🤖nvr,1.735226e+09,2278,47,eu_nvr,outro,GPT,NaN,Eu🤖nvr
2,1glbrnm,post,eu_nvr,1.730934e+09,2216,15,eu_nvr,outro,Transformers,NaN,eu_nvr
3,1hl2hd6,post,Eu_NVR,1.735003e+09,1653,23,eu_nvr,outro,GPT,NaN,Eu_NVR
4,18nbq7d,post,eu_nvr,1.703125e+09,1194,19,eu_nvr,outro,GPT,NaN,eu_nvr


In [22]:
import pandas as pd
import os

print("Carregando o dataset unificado para a limpeza final...")

# Caminho para o arquivo gerado na etapa anterior
UNIFIED_FILE = 'data/dataset_unificado_para_analise.csv'
FINAL_OUTPUT_FILE = 'data/dataset_final_pronto_para_llm.csv'

try:
    df = pd.read_csv(UNIFIED_FILE, dtype={'doc_id': str, 'post_id': str})
    print(f"Dataset carregado com sucesso! Contém {len(df)} linhas.")
except FileNotFoundError:
    print(f"Erro: Arquivo '{UNIFIED_FILE}' não encontrado. Verifique se a etapa anterior foi concluída com sucesso.")

Carregando o dataset unificado para a limpeza final...
Dataset carregado com sucesso! Contém 9337 linhas.


In [24]:
print("Iniciando a remoção de posts com a keyword 'agi' e seus respectivos comentários...")

# 1. Identificar os IDs dos posts que foram capturados pela keyword 'agi'
# Nós olhamos apenas os documentos do tipo 'post' para pegar o ID original da discussão.
post_ids_to_remove = set(df.loc[(df['keyword'] == 'agi') & (df['doc_type'] == 'post'), 'doc_id'])

if not post_ids_to_remove:
    print("Nenhum post encontrado com a keyword 'agi'. Nenhuma remoção necessária.")
else:
    print(f"Encontrados {len(post_ids_to_remove)} posts para remover (keyword 'agi').")

    # Armazena o número de linhas antes da remoção para relatório
    total_antes = len(df)

    # 2. Filtrar o DataFrame para remover os posts indesejados E seus comentários
    
    # Remove os posts cuja keyword é 'agi'
    df_final = df[~df['doc_id'].isin(post_ids_to_remove)]
    
    # Remove os comentários cujo post_id corresponde a um post removido
    # Esta é a parte crucial, garantindo que os comentários associados também sejam removidos.
    df_final = df_final[~df_final['post_id'].isin(post_ids_to_remove)]

    # 3. Salvar o DataFrame final
    os.makedirs('data', exist_ok=True)
    df_final.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"Dataset final salvo em: '{FINAL_OUTPUT_FILE}' ({len(df_final)} linhas)")

    print("\n--- Relatório da Remoção ---")
    print(f"Linhas antes da remoção: {total_antes}")
    print(f"Linhas após a remoção: {len(df_final)}")
    print(f"Total de {total_antes - len(df_final)} linhas removidas.")

Iniciando a remoção de posts com a keyword 'agi' e seus respectivos comentários...
Encontrados 235 posts para remover (keyword 'agi').
Dataset final salvo em: 'data/dataset_final_pronto_para_llm.csv' (8102 linhas)

--- Relatório da Remoção ---
Linhas antes da remoção: 9337
Linhas após a remoção: 8102
Total de 1235 linhas removidas.
